<a href="https://colab.research.google.com/github/Varad2510/CodeAlpha_Emotion_Recognition_from_Speech/blob/main/Emotion_Recognition_From_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Define Dataset Path
import zipfile
import os

DATASET_PATH = '/content/drive/My Drive/Speech Emotion Dataset/Speech_emotion_dataset.zip.zip'

# Step 3: Extract Dataset
extract_path = '/content/SpeechEmotionDataset'

with zipfile.ZipFile(DATASET_PATH, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Files extracted successfully!")
print("Extracted files:", os.listdir(extract_path))

# Step 4: Import Necessary Libraries
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

# Step 5: Data Preprocessing
def extract_features(file_path):
    try:
        # Load the audio file without resampling
        audio, sample_rate = librosa.load(file_path, sr=None)

        # Extract MFCC features
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

        # Return the mean of the MFCCs
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Prepare Dataset
features = []
labels = []

for folder in os.listdir(extract_path):
    folder_path = os.path.join(extract_path, folder)
    if os.path.isdir(folder_path):  # Check if it's a folder
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if file_path.endswith('.wav'):
                try:
                    data = extract_features(file_path)
                    if data is not None:  # Only append if features are extracted successfully
                        features.append(data)
                        labels.append(folder)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

print("Feature extraction completed!")

# Step 6: Encode Labels and Split Data
X = np.array(features)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# Step 7: Build the Model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model summary:")
model.summary()

# Step 8: Train the Model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

# Step 9: Evaluate the Model
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true_labels, y_pred_labels, target_names=label_encoder.classes_))

# Step 10: Save the Model
model.save('/content/drive/My Drive/SpeechEmotionModel.h5')
print("Model saved successfully to Google Drive!")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files extracted successfully!
Extracted files: ['Actor_05', 'Actor_02', 'Actor_09', 'Actor_20', 'Actor_14', 'Actor_04', 'Actor_07', 'Actor_17', 'Actor_19', 'Actor_01', 'Actor_13', 'Actor_06', 'Actor_15', 'Actor_24', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_03', 'Actor_08', 'audio_speech_actors_01-24', 'Actor_23', 'Actor_21', 'Actor_18', 'Actor_22', 'Actor_16']
Feature extraction completed!
Training data shape: (1152, 40)
Test data shape: (288, 40)
Model summary:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                      │ (None, 256)                 │          10,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 24)                  │           3,096 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 46,488 (181.59 KB)

 Trainable params: 46,488 (181.59 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.0374 - loss: 67.2327 - val_accuracy: 0.0417 - val_loss: 3.1780
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0360 - loss: 6.4910 - val_accuracy: 0.0417 - val_loss: 3.1784
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0345 - loss: 3.4359 - val_accuracy: 0.0417 - val_loss: 3.1786
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0451 - loss: 3.3698 - val_accuracy: 0.0417 - val_loss: 3.1790
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0383 - loss: 3.3061 - val_accuracy: 0.0417 - val_loss: 3.1794
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0580 - loss: 3.2052 - val_accuracy: 0.0417 - val_loss: 3.1797
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0507 - loss: 3.3029 - val_accuracy: 0.0417 - val_loss: 3.1801
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0375 - loss: 3.2619 - val_accuracy: 0.0417 - val_loss

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



              precision    recall  f1-score   support

    Actor_01       0.00      0.00      0.00        16
    Actor_02       0.00      0.00      0.00        13
    Actor_03       0.00      0.00      0.00        12
    Actor_04       0.00      0.00      0.00        12
    Actor_05       0.00      0.00      0.00        11
    Actor_06       0.00      0.00      0.00        10
    Actor_07       0.00      0.00      0.00        14
    Actor_08       0.00      0.00      0.00        13
    Actor_09       0.02      1.00      0.04         6
    Actor_10       0.00      0.00      0.00        12
    Actor_11       0.00      0.00      0.00        15
    Actor_12       0.00      0.00      0.00        10
    Actor_13       0.00      0.00      0.00        16
    Actor_14       0.00      0.00      0.00        14
    Actor_15       0.00      0.00      0.00        13
    Actor_16       0.00      0.00      0.00        17
    Actor_17       0.00      0.00      0.00         9
    Actor_18       0.00   